In [2]:
from collections import OrderedDict
from PyPDF2 import PdfFileWriter, PdfFileReader


def _getFields(obj, tree=None, retval=None, fileobj=None):
    """
    Extracts field data if this PDF contains interactive form fields.
    The *tree* and *retval* parameters are for recursive use.

    :param fileobj: A file object (usually a text file) to write
        a report to on all interactive form fields found.
    :return: A dictionary where each key is a field name, and each
        value is a :class:`Field<PyPDF2.generic.Field>` object. By
        default, the mapping name is used for keys.
    :rtype: dict, or ``None`` if form data could not be located.
    """
    fieldAttributes = {'/FT': 'Field Type', '/Parent': 'Parent', '/T': 'Field Name', '/TU': 'Alternate Field Name',
                       '/TM': 'Mapping Name', '/Ff': 'Field Flags', '/V': 'Value', '/DV': 'Default Value'}
    if retval is None:
        retval = OrderedDict()
        catalog = obj.trailer["/Root"]
        # get the AcroForm tree
        if "/AcroForm" in catalog:
            tree = catalog["/AcroForm"]
        else:
            return None
    if tree is None:
        return retval

    obj._checkKids(tree, retval, fileobj)
    for attr in fieldAttributes:
        if attr in tree:
            # Tree is a field
            obj._buildField(tree, retval, fileobj, fieldAttributes)
            break

    if "/Fields" in tree:
        fields = tree["/Fields"]
        for f in fields:
            field = f.getObject()
            obj._buildField(field, retval, fileobj, fieldAttributes)

    return retval


def get_form_fields(infile):
    infile = PdfFileReader(open(infile, 'rb'))
    fields = _getFields(infile)
    return OrderedDict((k, v.get('/V', '')) for k, v in fields.items())

In [18]:
def cleanAndReorderDict(oDict:OrderedDict) -> OrderedDict:
    for key,value in oDict.items():
        if('/True' in value):
            
        print(key,value)

In [4]:
from pprint import pprint

pdf_file_name = '../test-forms/labor_2hprotokoll_ncov2019_d_draft.pdf'
labDict=get_form_fields(pdf_file_name)
pprint(labDict)

OrderedDict([('Nasenrachenabstrich', '/True'),
             ('Mundrachenabstrich', ''),
             ('Sputum', ''),
             ('andere', '/True'),
             ('PLZ', '3033'),
             ('Arzt Auftraggeber',
              b'Arzt\rHinterkappellen Ring 111\r3032 Hinterkappellen\rTel 030 3'
              b'33 33 33\rFax 030 444 44 44'),
             ('Meldendes Labor',
              b'Labor Dieci\rSchwarztorstrasse 123\r3001 Bern\rTel 030 555 55 5'
              b'5\rFax 030 666 66 66'),
             ('Bemerkungen', 'keine Bemerkungen'),
             ('Patient-in_Familenname', 'Cow'),
             ('Patient-in_Vorname', 'Holy'),
             ('Nachweis_Tag', '21'),
             ('Nachweis_Monat', '03'),
             ('Nachweis_Jahr', '2020'),
             ('Andere_Entnahme', 'Augensekret'),
             ('Endotracheale_Absaugung', '/True'),
             ('Broncho-alveolaere_Lavage', ''),
             ('Blut_Serologie', ''),
             ('Patient-in_Strasse_Hausnummer', 'Weidestra

In [5]:
pdf_file_name = '../test-forms/ncov2019_d_optimized_draft.pdf'
meldDict=get_form_fields(pdf_file_name)
pprint(meldDict)

OrderedDict([('Kanton', 'BE'),
             ('Telefonnummer', '030 954 63 72'),
             ('Fieber', '/True'),
             ('Husten', '/True'),
             ('Atembeschwerden', '/True'),
             ('Durchfall', '/True'),
             ('ARDS', '/True'),
             ('Pneumonie', '/True'),
             ('Geburtstag_Tag', '12'),
             ('Geburtstag_Monat', '11'),
             ('Geburtstag_Jahr', '1989'),
             ('Andere Symptome', '/True'),
             ('Keine_Symptome', ''),
             ('Diabetes', '/Yes'),
             ('Herz-Kreislauf', '/Yes'),
             ('Immunsuppression', '/Yes'),
             ('Hypertonie', '/Yes'),
             ('Atemwegserkrankung', '/Yes'),
             ('Keine', ''),
             ('Krebs', '/Yes'),
             ('Andere_Symptome_Text', 'Kopfschmerz'),
             ('Andere', '/Yes'),
             ('Referenzlabor', '/Yes'),
             ('Andere_Grunderkrankungen', 'starke Raucherin'),
             ('Manifestation_Tag', '18'),
        

In [19]:
cleanAndReorderDict(meldDict)

Kanton BE
Telefonnummer 030 954 63 72
Fieber /True
Husten /True
Atembeschwerden /True
Durchfall /True
ARDS /True
Pneumonie /True
Geburtstag_Tag 12
Geburtstag_Monat 11
Geburtstag_Jahr 1989
Andere Symptome /True
Keine_Symptome 
Diabetes /Yes
Herz-Kreislauf /Yes
Immunsuppression /Yes
Hypertonie /Yes
Atemwegserkrankung /Yes
Keine 
Krebs /Yes
Andere_Symptome_Text Kopfschmerz
Andere /Yes
Referenzlabor /Yes
Andere_Grunderkrankungen starke Raucherin
Manifestation_Tag 18
Manifestation_Monat 03
Manifestation_Jahr 2020
Anderes_Labor 
Entnahme_Tag 19
Entnahme_Monat 03
Entnahme_Jahr 2020
IPS 
Beatmung 
Anderes_Labor_text 
Eintrittsdatum_Tag 20
Eintrittsdatum_Monat 03
Eintrittsdatum_Jahr 2020
TF /Yes
Spital b'Inselspital Bern\rTel 030 548 44 44'
TD_Tag 22
TD_Monat 03
TD_Jahr 2020
Exposition_Tag 13
Exposition_Monat 03
Exposition_Jahr 2020
Von_Tag 06
Von_Monat 03
Von_Jahr 2020
Bemerkungen Keine Bemerkungen
Bis_Tag 16
Bis_Monat 03
Bis_Jahr 2020
Andere_Kontakte_text 
Datum_Tag 23
KA_Datum_Tag 19
Datum_M

In [15]:
import pandas as pd

df=pd.DataFrame(meldDict,columns=meldDict.keys(),index=[0])
df.head()

,Kanton,Telefonnummer,Fieber,Husten,Atembeschwerden,Durchfall,ARDS,Pneumonie,Geburtstag_Tag,Geburtstag_Monat,...,Exposition-Land_Text,Exposition-Ort_Text,Ausland_Aufenthalt_Dauer_unbekannt,Reise_mit_Infektion,Kontakt_Familie,Kontakt_Medizinpersonal,Kontakt_Schule,Kontakt_Arbeit,Kontakt_Andere,Kantonsarzt_Massnahmen_Text
0,BE,030 954 63 72,/True,/True,/True,/True,/True,/True,12,11,...,Italien,Bergamo,,/True,/Medizin-Pflege#20Personal,,/Schule,,,Isolation
